In [39]:
import pandas as pd
import numpy as np
import keras
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [37]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Conv2D, MaxPool2D
from keras import regularizers, optimizers

In [29]:
train_data_path = 'data/train/train/'
test_data_path = 'data/test/test/'
#wav_path = 'data/wav/'

# Ham doi duoi tu wav sang png
def append_ext(fn):
    return fn.replace(".wav",".png")

In [5]:
traindf = pd.read_csv('data/train.csv',dtype=str)
testdf = pd.read_csv('data/test.csv',dtype=str)
traindf["slice_file_name"] = traindf["slice_file_name"].apply(append_ext)
testdf["slice_file_name"] = testdf["slice_file_name"].apply(append_ext)

In [13]:
traindf
#print(testdf)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.png,100032,0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.png,100263,58.5,62.5,1,5,2,children_playing
2,100263-2-0-121.png,100263,60.5,64.5,1,5,2,children_playing
3,100263-2-0-126.png,100263,63,67,1,5,2,children_playing
4,100263-2-0-137.png,100263,68.5,72.5,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99157-9-0-15.png,99157,12.428661,16.428661,2,4,9,street_music
8728,99157-9-0-16.png,99157,12.928661,16.928661,2,4,9,street_music
8729,99157-9-0-22.png,99157,15.928661,19.928661,2,4,9,street_music
8730,99157-9-0-24.png,99157,16.928661,20.928661,2,4,9,street_music


In [10]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [30]:
train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_data_path,
    x_col="slice_file_name",
    y_col="class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64)
)

valid_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=train_data_path,
    x_col="slice_file_name",
    y_col="class",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64)
)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 4732 invalid image filename(s) in x_col="slice_file_name". These filename(s) will be ignored.
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 4732 invalid image filename(s) in x_col="slice_file_name". These filename(s) will be ignored.
  warnings.warn(


Found 3000 validated image filenames belonging to 10 classes.
Found 1000 validated image filenames belonging to 10 classes.


array([], dtype=int32)

In [49]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.RMSprop(learning_rate=0.0005, epsilon=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 62, 62, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 64)        0         
                                                        

In [50]:
# Tinh so buoc trong 1 epoch khi train
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
# Tinh so buoc trong 1 epoch khi val
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

# Train model
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30,verbose=1
)

Epoch 1/30


C:\Users\Admin\AppData\Local\Temp/ipykernel_2424/3119554097.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


93/93 [==============================] - 33s 351ms/step - loss: 2.1218 - accuracy: 0.1971 - val_loss: 1.9996 - val_accuracy: 0.2571
Epoch 2/30
93/93 [==============================] - 32s 348ms/step - loss: 1.8338 - accuracy: 0.3080 - val_loss: 2.0701 - val_accuracy: 0.2208
Epoch 3/30
93/93 [==============================] - 32s 343ms/step - loss: 1.4903 - accuracy: 0.4727 - val_loss: 2.1252 - val_accuracy: 0.3417
Epoch 4/30
93/93 [==============================] - 31s 336ms/step - loss: 1.1966 - accuracy: 0.5920 - val_loss: 1.9014 - val_accuracy: 0.4073
Epoch 5/30
93/93 [==============================] - 32s 343ms/step - loss: 0.9907 - accuracy: 0.6678 - val_loss: 1.7574 - val_accuracy: 0.4355
Epoch 6/30
93/93 [==============================] - 34s 366ms/step - loss: 0.8267 - accuracy: 0.7268 - val_loss: 1.9721 - val_accuracy: 0.4567
Epoch 7/30
93/93 [==============================] - 32s 344ms/step - loss: 0.7079 - accuracy: 0.7551 - val_loss: 1.7599 - val_accuracy: 0.4688
Epoch 8/30

In [35]:
model.save("model.h5")

In [48]:
keras.backend.clear_session()

In [45]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
modelvgg = Sequential()
modelvgg.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
modelvgg.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
modelvgg.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
modelvgg.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
modelvgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
modelvgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
modelvgg.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
modelvgg.add(Dropout(0.5))
modelvgg.add(Flatten())
modelvgg.add(Dense(units=4096,activation="relu"))
modelvgg.add(Dropout(0.5))
modelvgg.add(Dense(units=4096,activation="relu"))
modelvgg.add(Dropout(0.5))
modelvgg.add(Dense(units=10, activation="softmax"))
modelvgg.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
modelvgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [46]:
# Tinh so buoc trong 1 epoch khi train
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
# Tinh so buoc trong 1 epoch khi val
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

# Train model
modelvgg.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,verbose=1
)

Epoch 1/50


C:\Users\Admin\AppData\Local\Temp/ipykernel_2424/358152474.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  modelvgg.fit_generator(generator=train_generator,


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/flatten/Reshape' defined at (most recent call last):
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\tornado\platform\asynci